# TP 6 LSTMs & GRU: génération de séquences

_Ismaël Bonneau_

Ce notebook sert uniquement à présenter nos résultats, et les bouts de code intéressants dans le cadre de ce rapport. L'intégralité du code est contenu dans le fichier tp6.py.

## But

Nous avons dans le notebook précédent codé un RNN from scratch et nous l'avons appliqué sur une tâche de génération de séquence. Nous allons maintenant reprendre cette tâche et utiliser à la place du RNN un LSTM et un GRU, codés from scratch.

<img src="../images/rnn_vs_lstm.png" width="500">

## Données

Le jeu de données suggéré dans l'énoncé est un discours de Donald Trump. Cependant, il est beaucoup plus drôle de travailler sur le script du seigneur des anneaux. 

In [1]:
import numpy as np
import torch
import torch.nn as nn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set()

%matplotlib inline
%load_ext autoreload
%autoreload 2

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
bestmovieever = pd.read_csv("../TP4/data/lord-of-the-rings-data/lotr_scripts.csv")

### Récupérons les lignes de dialogue des 3 films du seigneur des anneaux. Nous allons garder uniquement les répliques de Gandalf.

In [3]:
characters = ["GANDALF", "\xa0GANDALF", "GAN DALF", 'GANDALF VOICEOVER']
bestmovieever[bestmovieever["char"].isin(characters)].head()

,Unnamed: 0,char,dialog,movie
34,34,GANDALF,Now come the days of the King. May they be...,The Return of the King
68,68,GANDALF,Hobbits!,The Return of the King
72,72,GANDALF,"Be careful. Even in defeat, Saruman is dangero...",The Return of the King
74,74,GANDALF,"No, we need him alive. We need him to talk.",The Return of the King
78,78,GANDALF,Your treachery has already cost many lives. Th...,The Return of the King


### On nettoie un peu les lignes de dialogue:

On sépare en phrases, et on élimine les phrases trop courtes (comme "no.").

In [4]:
from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_multiple_whitespaces, strip_tags, strip_numeric
import re

CUSTOM_FILTERS = [strip_tags, strip_multiple_whitespaces, strip_numeric]

hey = list(bestmovieever[bestmovieever["char"].isin(characters)]["dialog"])
gandalf = []
for d in hey:
    mdr = " ".join(preprocess_string(d, CUSTOM_FILTERS))
    for ahaha in mdr.split("."):
        if ahaha != "":
            x = ahaha.strip()
            if ("!" != x[-1]) and ("?" != x[-1]):
                x = x+"."
            x = re.sub(r'^ , ', '', x)
            x = re.sub(r'^, ', '', x)
            x = re.sub(r'^,', '', x)
            if len(x) > 8:
                gandalf.append(x)

In [5]:
gandalf[18:34]

["End? No, the journey doesn't end here.",
 'Death is just another path, one that we all must take.',
 'The grey rain curtain of this world rolls back and all turns to silvered glass.',
 'And then you see it.',
 'White shores and beyond, a far green country under a swift sunrise.',
 "No it isn't.",
 "Saruman! You were deep in the enemy's counsel.",
 'Tell us what you know!',
 'Send word to all our allies and to every corner of Middle Earth that still stands free.',
 'The enemy moves against us.',
 'We need to know where he will strike.',
 'Peregrin Took.',
 "I'll take that my lad! Quickly now!",
 'Retreat! The city is breached.',
 'Fall back to the second level.',
 'Get the women and children out.']

### Nous allons travailler au niveau des caractères. 

il faut encoder chaque caractère sous forme d'id numérique (entier). On associe donc à chaque caractère un entier. le 0 servira au padding, et l'id _nb caractères+1_ servira à signaler la fin de séquence.

In [6]:
import string
import unicodedata

max_len = len(gandalf[np.argmax([len(s) for s in gandalf])]) + 1 # +1 pour la fin de séquence
print("longueur max. de sequence: {}".format(max_len))

LETTRES = set()
for phrase in gandalf:
    LETTRES.update(list(phrase))
LETTRES.update(list(string.ascii_letters))
id2lettre = dict(zip(range(1, len(LETTRES)+1),LETTRES))
id2lettre[0]= '' # NULL CHARACTER for padding
id2lettre[len(LETTRES)+1] = "EOF"
lettre2id = dict(zip(id2lettre.values(), id2lettre.keys()))

def normalize(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if c in LETTRES)
def string2code(s):
    base = [lettre2id[c] for c in normalize(s)] + [lettre2id["EOF"]]
    if len(base) < max_len:
        padding = [0] * (max_len - len(base))
    else:
        padding = []
    return base + padding + [0]
def code2string(t):
    if type(t) != list:
        t = t.tolist()
    return ''.join(id2lettre[i] for i in t)

longueur max. de sequence: 188


#### Exemple de phrase transformée en suite d'entiers:

On voit que l'on a ajouté du padding (des 0) à la fin des phrases pour avoir des sequences de longueur identique.

In [9]:
print(gandalf[38])
print(string2code(gandalf[38]))

Go back to the abyss! Fall into the nothingness that awaits you and your master!
[3, 12, 24, 45, 27, 13, 29, 24, 51, 12, 24, 51, 38, 55, 24, 27, 45, 50, 47, 47, 37, 24, 35, 27, 22, 22, 24, 17, 6, 51, 12, 24, 51, 38, 55, 24, 6, 12, 51, 38, 17, 6, 36, 6, 55, 47, 47, 24, 51, 38, 27, 51, 24, 27, 1, 27, 17, 51, 47, 24, 50, 12, 34, 24, 27, 6, 5, 24, 50, 12, 34, 39, 24, 33, 27, 47, 51, 55, 39, 37, 63, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


#### Pas besoin de one-hot encoding car on va utiliser le module nn.Embeddings

On utilise _np.exand_dims_ pour avoir une matrice en 3 dimensions _nb sequences x longueur sequence x dimension données

In [10]:
data = np.array([np.array(string2code(s)) for s in gandalf])
data = np.expand_dims(data, axis=2)
print(data.shape)

(417, 189, 1)


Le GRU que nous avons codé attend en entrée des matrices de taille _longueur sequence x nb sequences (batch) x dimension données_. On va donc swaper les dimensions 0 et 1 de notre array de séquences:

In [11]:
data = np.swapaxes(data, 0, 1)
print(data.shape)

(189, 417, 1)


Et voilà les donneés prêtes à être ingérées par le modèle!

## Modèle

Cette fois-ci, on introduit la supervision à chaque étape: au lieu de devoir encoder correctement une classe dans l'état ${h_T}$, le modèle doit réussir à produire ${x_t}$ à partir de ${h_{t-1}}$ à chaque étape. Chaque "cellule" du GRU déplié doit donc appliquer un décodeur sur l'état ${h_{t-1}}$ qu'elle reçoit en entrée.

Le fait que l'on utilise des caractères rajoute une difficulté supplémentaire: il va falloir en même temps calculer une projection des caractères dans un espace continu (embedding).

L'entrainement va fonctionner comme ceci:

- On traite le batch de sequences avec une passe de forward.
- On récupère l'historique des ${h_1}$, ..., ${h_{T-1}}$ calculés par le réseau.
- On décode l'historique.
- On calcule la loss (cross entropy) sur les valeurs décodées qui doivent correspondre à ${h_2}$, ..., ${h_{T}}$
- On masque la loss aux endroits qui correspondent au padding pour chaque batch, et on applique la backward propagation.


<img src="../images/gru.png" width="600">

In [19]:
class GRU(nn.Module):
    """docstring for GRU"""
    def __init__(self, input_dim, latent_dim):
        super(GRU, self).__init__()
        self.latent_dim = latent_dim
        
        self.W_update_x = nn.Linear(input_dim, latent_dim)
        self.W_update_h = nn.Linear(latent_dim, latent_dim)
        
        self.W_reset_x = nn.Linear(input_dim, latent_dim)
        self.W_reset_h = nn.Linear(latent_dim, latent_dim)
        
        self.W_x = nn.Linear(input_dim, latent_dim)
        self.W_h = nn.Linear(latent_dim, latent_dim)
        self.sigmoid = nn.Sigmoid()
        self.tanh = nn.Tanh()

    def one_step(self, x, h):
        """ one step for one Xt and Ht-1 """
        zt = self.sigmoid(self.W_update_x(x) + self.W_update_h(h))
        rt = self.sigmoid(self.W_reset_x(x) + self.W_reset_h(h))
        ht = (1 - zt) * h + zt * self.tanh(self.W_x(x) + self.W_h(rt * h))
        return ht

    def forward(self, x, h=None):
        """ forward on the whole sequence """
        historique = []
        if h is None:
            ht = torch.zeros(x.size()[1], self.latent_dim)
        for xt in x:
            # ht: (batch x latent)
            ht = self.one_step(xt, ht)
            historique.append(ht)
        return historique

In [68]:
class Character_level_encoder(torch.nn.Module):
    """ projette les caractères dans un embedding """
    def __init__(self, vocab_size, embedding_dim):
        super(Character_level_encoder, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim) #apprendre des embeddings de caractère en même temps
    def forward(self, inputs):
        embeds = self.embeddings(inputs)
        return embeds

In [14]:
class Decoder_cell(torch.nn.Module):
    """ decode un etat caché """
    def __init__(self, latent, dim):
        super(Decoder_cell, self).__init__()
        # 1st param: nmbr de caractères, 2nd param: embedding dim
        self.W = nn.Sequential(nn.Linear(latent, 16),
                               nn.Tanh(),
                               nn.Linear(16, dim), nn.Softmax(dim=2))
        # doit pouvoir produire une classe à partir d'un état caché
    def forward(self, h):
        """ """
        return self.W(h)